In [2]:
import pandas as pd
import numpy as np
import igraph as ig

import plotly.express as px

from matplotlib import pyplot as plt

from helpers import flatten_list, applyParallel

import nltk
from nltk.corpus import stopwords

In [3]:
random_state = 42
stopwords = stopwords.words('english')

In [4]:
genome_tag_vec = pd.read_csv("cleaned/genome_tag_vec.csv")
imdb_data = pd.read_csv("cleaned/imdb_data.csv")
movies = pd.read_csv("cleaned/movies.csv")
# tags = pd.read_csv("data/tags.csv")
test = pd.read_csv("data/test.csv")
train = pd.read_csv("data/train.csv")

In [5]:
genome_tag_vec.head()

,movieId,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
0,1,0.02875,0.02375,0.06250,0.07575,0.14075,0.14675,0.06350,0.20375,0.2020,...,0.04050,0.01425,0.03050,0.03500,0.14125,0.05775,0.03900,0.02975,0.08475,0.02200
1,2,0.04125,0.04050,0.06275,0.08275,0.09100,0.06125,0.06925,0.09600,0.0765,...,0.05250,0.01575,0.01250,0.02000,0.12225,0.03275,0.02100,0.01100,0.10525,0.01975
2,3,0.04675,0.05550,0.02925,0.08700,0.04750,0.04775,0.04600,0.14275,0.0285,...,0.06275,0.01950,0.02225,0.02300,0.12200,0.03475,0.01700,0.01800,0.09100,0.01775
3,4,0.03425,0.03800,0.04050,0.03100,0.06500,0.03575,0.02900,0.08650,0.0320,...,0.05325,0.02800,0.01675,0.03875,0.18200,0.07050,0.01625,0.01425,0.08850,0.01500
4,5,0.04300,0.05325,0.03800,0.04100,0.05400,0.06725,0.02775,0.07650,0.0215,...,0.05350,0.02050,0.01425,0.02550,0.19225,0.02675,0.01625,0.01300,0.08700,0.01600


In [6]:
imdb_data.head()

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,30000000.0,toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,65000000.0,board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,25000000.0,boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,16000000.0,black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,30000000.0,fatherhood|doberman|dog|mansion


In [7]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


In [8]:
# tags.head()

In [9]:
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [10]:
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [12]:
train.rating.std()

1.0611240700473945

In [10]:
distrib = train.rating.value_counts(normalize=True)

fig = px.bar(distrib, color=distrib.index)

fig.show()

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler, MinMaxScaler, RobustScaler

from sklearn.metrics.pairwise import cosine_similarity

from scipy import sparse

from sklearn.decomposition import TruncatedSVD

from sklearn.cluster import KMeans

In [12]:
full_combine = (
    movies
    .merge(imdb_data,      on="movieId", how="left")
    .merge(genome_tag_vec, on="movieId", how="left")
    .set_index("movieId", drop=True)
)

normal_year_mean = full_combine[full_combine.year != 0].year.mean()
full_combine['year'].replace(0, normal_year_mean, inplace=True)

# movies.year.fillna(int(movies.year.median()), inplace=True)
# movies.genres.fillna("<unknown>", inplace=True)
full_combine.title = full_combine.title.str.strip().str.replace(r"(.*), The$", r"The \1", regex=True)
full_combine.title_cast.fillna("", inplace=True)
full_combine.director.fillna("", inplace=True)
full_combine.runtime.fillna(int(full_combine.runtime.median()), inplace=True)
full_combine.budget.fillna(int(full_combine.budget.median()), inplace=True)
full_combine.plot_keywords.fillna("", inplace=True)

full_combine['cast_size'] = full_combine.title_cast.str.split('|').apply(len)
full_combine['genre_count'] = full_combine.genres.str.split('|').apply(len)

movie_groups = train.set_index('movieId', drop=True).groupby('movieId')
full_combine['rating_mean'] = movie_groups.rating.mean()
full_combine['rating_std'] = movie_groups.rating.std().fillna(0)

q3 = movie_groups.rating.quantile(0.75)
q1 = movie_groups.rating.quantile(0.25)

full_combine['rating_iqr'] =  q3 - q1
full_combine['rating_count'] = movie_groups.apply(len)

full_combine.fillna(0, inplace=True)

full_combine['movieId'] = full_combine.index.values
full_combine.reset_index(inplace=True, drop=True)
full_combine.head()

,title,genres,year,title_cast,director,runtime,budget,plot_keywords,007,007 (series),...,wwii,zombie,zombies,cast_size,genre_count,rating_mean,rating_std,rating_iqr,rating_count,movieId
0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,30000000.0,toy|rivalry|cowboy|cgi animation,0.02875,0.02375,...,0.02975,0.08475,0.02200,15,5,3.889971,0.924249,1.0,23062.0,1
1,Jumanji,Adventure|Children|Fantasy,1995.0,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,65000000.0,board game|adventurer|fight|game,0.04125,0.04050,...,0.01100,0.10525,0.01975,15,3,3.263414,0.966744,1.0,9654.0,2
2,Grumpier Old Men,Comedy|Romance,1995.0,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,25000000.0,boat|lake|neighbor|rivalry,0.04675,0.05550,...,0.01800,0.09100,0.01775,15,2,3.132325,1.016368,1.0,4727.0,3
3,Waiting to Exhale,Comedy|Drama|Romance,1995.0,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,16000000.0,black american|husband wife relationship|betra...,0.03425,0.03800,...,0.01425,0.08850,0.01500,15,3,2.878099,1.073617,2.0,968.0,4
4,Father of the Bride Part II,Comedy,1995.0,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,30000000.0,fatherhood|doberman|dog|mansion,0.04300,0.05325,...,0.01300,0.08700,0.01600,15,1,3.059165,0.995982,1.5,4648.0,5


In [13]:
features = []
full_combine.columns

Index(['title', 'genres', 'year', 'title_cast', 'director', 'runtime',
       'budget', 'plot_keywords', '007', '007 (series)',
       ...
       'wwii', 'zombie', 'zombies', 'cast_size', 'genre_count', 'rating_mean',
       'rating_std', 'rating_iqr', 'rating_count', 'movieId'],
      dtype='object', length=1143)

In [14]:
# title_vectrz = CountVectorizer(min_df=25, ngram_range=(1, 3), stop_words=stopwords, norm=None)
# title_vec = title_vectrz.fit_transform(full_combine.title)
# print("Title Tokens:", len(title_vectrz.get_feature_names()))
# features.extend(title_vectrz.get_feature_names())

In [14]:
genre_vectrz = CountVectorizer(token_pattern=r"[A-z\-]+", min_df=2)
genre_vec = genre_vectrz.fit_transform(full_combine.genres)
print("Genre Tokens:", len(genre_vectrz.get_feature_names()))
features.extend(genre_vectrz.get_feature_names())

Genre Tokens: 20


In [15]:
cast_vectrz = CountVectorizer(token_pattern=r"[^\|]+", min_df=20)
cast_vec = cast_vectrz.fit_transform(full_combine.title_cast)
print("Cast Tokens:", len(cast_vectrz.get_feature_names()))
features.extend(cast_vectrz.get_feature_names())

Cast Tokens: 817


In [16]:
director_vectrz = CountVectorizer(token_pattern=r".+", min_df=10, stop_words=['see full summary'])
director_vec = director_vectrz.fit_transform(full_combine.director)
print("Director Tokens:", len(director_vectrz.get_feature_names()))
features.extend(director_vectrz.get_feature_names())

Director Tokens: 30


In [17]:
plot_vectrz = CountVectorizer(token_pattern=r"[^\|]+", min_df=10, stop_words=stopwords)
plot_vec = plot_vectrz.fit_transform(full_combine.plot_keywords)
print("Plot KW Tokens:", len(plot_vectrz.get_feature_names()))
features.extend(plot_vectrz.get_feature_names())

Plot KW Tokens: 1001


In [18]:
genome_features = [col for col in genome_tag_vec.columns if col != 'movieId']
features.extend(genome_features)
sparse_genome = sparse.csr_matrix(full_combine[genome_features])

In [19]:
extra_features = ["year", "runtime", "budget", "rating_mean", "rating_std", "rating_iqr", "rating_count"]
features.extend(extra_features)

extra_features = full_combine[extra_features]

scaler = MinMaxScaler()
transformed = scaler.fit_transform(extra_features)
std_extra_sparse = sparse.csr_matrix(transformed)

tfidf_vecs = sparse.hstack([
    # title_vec,
    genre_vec,
    cast_vec,
    director_vec,
    plot_vec,
]).tocsr()

vecs = sparse.hstack([tfidf_vecs, sparse_genome, std_extra_sparse])
norm = Normalizer(copy=True)
norm_vecs = norm.transform(vecs)
norm_vecs.shape

(62423, 3003)

In [20]:
a = norm_vecs.getrow(17067).toarray().flatten()
b = norm_vecs.getrow(14628).toarray().flatten()

f = pd.DataFrame(index=features)
f['a'] = a
f['b'] = b
f['diff'] = np.abs(f.a - f.b)

f[f.a != f.b].sort_values('diff', ascending=False)

,a,b,diff
sam rockwell,0.000000,0.107183,1.071829e-01
based on comic book,0.000000,0.107183,1.071829e-01
thriller,0.000000,0.107183,1.071829e-01
don cheadle,0.000000,0.107183,1.071829e-01
jon favreau,0.000000,0.107183,1.071829e-01
...,...,...,...
ocean,0.004988,0.004984,4.051464e-06
rabbits,0.001175,0.001179,3.940940e-06
christianity,0.000887,0.000884,3.039553e-06
animals,0.006595,0.006592,3.037643e-06


In [21]:
def most_similar_to(movieId):
    movie_idx = movies[movies['movieId'] == movieId].index[0]

    return norm_vecs.dot(norm_vecs.getrow(movie_idx).transpose())

avengers = 89745
avengers2 = 122892
amazing_spiderman = 95510
fault_in_stars = 111921

mst = most_similar_to(avengers).toarray().flatten()
n = 10
top_n = mst.argsort(axis=0)[-n:][::-1]
bot_n = mst.argsort(axis=0)[:n][::-1]

for similar_idx in top_n:
    title = full_combine.iloc[similar_idx].title
    sim = round(mst[similar_idx], 10)
    print(sim, similar_idx, title)

print('-' * 20)

for similar_idx in bot_n:
    title = full_combine.iloc[similar_idx].title
    sim = mst[similar_idx]
    print(sim, title)
# most_similar = most_similar_to(avengers).toarray().argmax()
# movie_data.iloc[most_similar]

1.0 17067 The Avengers
0.8951058359 12324 Iron Man
0.8915574534 25058 Avengers: Age of Ultron
0.8836220293 25061 Ant-Man
0.8812012854 14628 Iron Man 2
0.8804854623 21348 Captain America: The Winter Soldier
0.8711449397 25072 Doctor Strange
0.8643691094 25071 Captain America: Civil War
0.8628606915 16312 Thor
0.8589632293 25070 Guardians of the Galaxy 2
--------------------
0.017348902078642935 Miniature Theatre
0.017081294194473868 Caicedo (with Pole)
0.016067734717298793 Apparitions
0.016067734717298793 The Oracle of Delphi
0.016067734717298793 Jupiter's Thunderballs
0.015262073614173423 Je vous aime
0.014942000994070915 Diabolical Pickpocket
0.01477810705079454 Explosion of a Motor Car
0.012176408254379163 Fire Rescue Scene
0.008480747746019753 Athlete Swinging a Pick


In [22]:
movie_indexes = full_combine.movieId.copy()
experiment = train.copy().drop('timestamp', axis=1)

mid_to_idx = pd.Series(movie_indexes.index.values, index=movie_indexes).to_dict()

In [23]:
def remove_diag(x):
    x_no_diag = np.ndarray.flatten(x)
    x_no_diag = np.delete(x_no_diag, range(0, len(x_no_diag), len(x) + 1), 0)
    x_no_diag = x_no_diag.reshape(len(x), len(x) - 1)
    return x_no_diag

In [24]:
mean_movie_rating_by_movie_id = train.groupby('movieId').rating.mean()

In [25]:
# combined_data = pd.concat([train, test])
# combined_data.drop("timestamp", axis=1, inplace=True)
# combined_data.sort_values(['userId', 'rating'], inplace=True)
# combined_data.reset_index(inplace=True, drop=True)
# combined_data

In [26]:
def predict_ratings(data):
    history = data[data.rating.notna()]
    to_predict = data[data.rating.isna()]

    if len(history) == 1:
        print(data.userId.unique())
        mean_guesses = mean_movie_rating_by_movie_id.loc[to_predict.movieId.values].values
        print(mean_guesses)
        to_predict['pred'] = mean_guesses
        return pd.concat([history, to_predict])

    if len(to_predict) == 0:
        return data

    idxs = [mid_to_idx[i] for i in data.movieId]

    historic_ratings = history.rating.values

    similarity_matrix = remove_diag(cosine_similarity(norm_vecs[idxs, :]))
    sim_matrix = similarity_matrix[len(history):, :len(history)]
    
    sim_totals = sim_matrix.sum(axis=1)
    ratings = np.tile(history.rating, (len(to_predict.rating),1))
    weighted_sums = np.einsum('ij,ij->i', sim_matrix, ratings)

    to_predict['pred'] = weighted_sums / sim_totals
    
    return pd.concat([history, to_predict])

In [27]:
# %%time
# parallel_preds = applyParallel(combined_data.groupby("userId"), predict_ratings)

In [28]:
experiment = train.copy()
experiment.drop('timestamp', inplace=True, axis=1)

experiment['original'] = experiment['rating']
experiment['rating'] = experiment.rating.sample(frac=0.85)

In [29]:
%%time
experimental_preds = applyParallel(experiment.groupby("userId"), predict_ratings)

CPU times: user 6min 12s, sys: 41.9 s, total: 6min 54s
Wall time: 15min 35s


In [30]:
p = experimental_preds[experimental_preds.pred.notna()].copy()
p['err'] = np.abs(p.original - p.pred)

In [31]:
mae_by_movie = p.groupby('movieId').err.mean().sort_values(ascending=False)
extreme_movies = pd.DataFrame(mae_by_movie[mae_by_movie > 3.])
extreme_movies['count'] = train.groupby('movieId').apply(len)
extreme_movies['rating_mean'] = full_combine.set_index('movieId', drop=True).rating_mean
extreme_movies

,err,count,rating_mean
movieId,,,
193267,4.338793,3,1.666667
5565,4.109974,3,1.500000
199442,4.039009,2,1.750000
199452,4.038053,3,0.500000
133547,4.026578,3,2.333333
...,...,...,...
172459,3.034312,6,2.250000
76836,3.031792,5,2.400000
78642,3.010391,2,1.000000


In [32]:
np.sqrt(np.mean(np.power(p.original - p.pred, 2)))

0.9603667186227086

In [33]:
mae_by_user = p.groupby('userId').err.mean().sort_values(ascending=False)
extreme_users = mae_by_user[mae_by_user > 3.]
extreme_users

userId
156196    4.295730
52594     4.256298
71695     4.230000
100662    4.134870
151339    4.131356
            ...   
85754     3.014344
159372    3.011203
161610    3.010237
125776    3.003663
48981     3.003604
Name: err, Length: 254, dtype: float64

In [35]:
def make_submission_file(df, filename="submission.csv"):
    df.reset_index(drop=True)
    df.drop(df[df.rating.notna()].index, inplace=True)
    df["rating"] = df["pred"]
    df["Id"] = df.userId.astype(str) + "_" + df.movieId.astype(str)
    df[["Id", "rating"]].to_csv(filename, index=False, chunksize=100_000)

In [40]:
%%time

community = dict()

users = len(train.userId.unique())

movie_community = train.set_index('movieId', drop=True).drop("timestamp", axis=1)

for idx, (uid, group) in enumerate(train.groupby("userId")):
    movies = group.movieId.unique()
    community[uid] = movie_community.loc[movies].userId.unique()

    if ((idx + 1) % 1_000) == 0:
        print("[progress]", round(idx / len(users) * 100.0, 2), "%")

KeyboardInterrupt: 

,userId,rating,timestamp
movieId,,,
57669,5163,4.0,1518349992
5,106343,4.5,1206238739
5459,146790,5.0,1076215539
32296,106362,2.0,1423042565
366,9041,3.0,833375837
...,...,...,...
99114,136395,5.0,1521235092
553,140078,3.0,1002580977
56782,154807,4.0,1227674807
